# **Parameter Optimisation for GITT** 

This notebook is created to provide an example for ```GITT``` optimisation problem. Diffisuion Coefficient is optimised in this notebook. Only voltage will be considered for optimisation in this example.
 The workflow is as follows:


In [1]:
%pip install git+https://github.com/paramm-team/pybamm-param.git -q
import pybamm
import pbparam
import pandas as pd
import os
os.chdir(os.path.join(pbparam.__path__[0], "input", "data"))

We start by importing the example experimental dataset which corresponds to the [Brosa Planella et al. (2021) article](https://www.sciencedirect.com/science/article/pii/S0013468621008148).

In [3]:
data = pd.read_csv("gitt.csv")

In [4]:
data

,Time [s],Voltage [V],Current [A],Negative electrode potential [V],Positive electrode potential [V]
0,0.00,3.9590,5.000229,0.1871,3.7639
1,1.00,3.9536,4.999619,0.1855,3.7578
2,2.00,3.9498,4.999619,0.1842,3.7554
3,3.00,3.9469,5.000381,0.1836,3.7530
4,4.00,3.9445,4.999466,0.1828,3.7513
...,...,...,...,...,...
146,146.00,3.8766,5.000687,0.1610,3.7053
147,147.00,3.8764,4.999924,0.1610,3.7052
148,148.00,3.8761,4.999924,0.1611,3.7052
149,149.00,3.8759,5.000839,0.1608,3.7050


PyBaMM model to be used for parameter optimisation needs to be defined:

In [5]:
model = pbparam.WeppnerHuggins()

Rest of the PyBaMM simulation requirements and initial parameters defined here:

In [6]:
param = model.default_parameter_values # Initial parameters to be used.

# Update some parameters to match the experimental setup
param.update(
    {
        "Reference OCP [V]": 4.2,
        "Derivative of the OCP wrt stoichiometry [V]": -1,
        "Positive electrode diffusivity [m2.s-1]": 4e-15
    },
    check_already_exists=False,
)

simulation = pybamm.Simulation(model, parameter_values=param)

PyBaMM-param needs a dictionary for parameters to be optimised in ```{"parameter name": (initial guess, (lower bound, upper bound))}```

In [6]:
param_optimise = {
    "Positive electrode diffusivity [m2.s-1]": (5e-14, (2.06e-16, 2.06e-12)),
}

In [7]:
# Since this is a parameter optimisation with fitting data into experimental data, optimisation problem is DataFit.
opt = pbparam.GITT(simulation, data)

Then, Optimisation method should be defined in this step. DiferentialEvolution(https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html) or ScipyMinimize(https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)

can be used to minimise cost function. For faster results, ScipyMinimize with ```"Nelder-Mead"``` method can be used. However, ScipyDifferentialEvolution is more robust and has provided lower value for most of the cases. 

In [8]:
# optimiser = pbparam.ScipyDifferentialEvolution(
    # extra_options={"workers": 4, "polish": True, "updating": "deferred", "disp": True}
# )
optimiser = pbparam.ScipyMinimize(method="Nelder-Mead")

After providing everything for optimisation algorithm, it can be performed using `optimise(optimisation_problem)`

In [9]:
result = optimiser.optimise(opt)
# optimised values of parameters and function values can be printed as below.
print(result)


             Optimal values: [5.e-14]
        Cost function value: 370.67212101452776
                 Solve time: 79.609 ms
                    Message: Optimization terminated successfully.
        


```plot()``` function can be used to present result graphically.

In [10]:
result.plot()

AttributeError: 'GITT' object has no attribute 'simulation'